### Prefix

In [1]:
import sys
sys.path.insert(0,'..')

### Imports

Following the guide from https://huggingface.co/docs/transformers/tasks/language_modeling. 

In [8]:
from superhf import skeleton
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, GPTNeoForCausalLM
import numpy as np
import torch
import seaborn as sns

## Load the Models
1. Reward Model = (DeBERTa-v3-base (~730mb)
2. SuperHF Model = GPT-Neo-1.3B (~1.3gb)

In [4]:
reward_name = "OpenAssistant/reward-model-deberta-v3-base"
reward_model, reward_tokenizer = AutoModelForSequenceClassification.from_pretrained(reward_name), AutoTokenizer.from_pretrained(reward_name)

In [9]:
assistant_name = "EleutherAI/gpt-neo-1.3B"
assistant_model = GPTNeoForCausalLM.from_pretrained(assistant_name)
assistant_tokenizer = AutoTokenizer.from_pretrained(assistant_name)
# this is the gpt2 tokenizer.

### Sanity Check GPT-Neo-1.3B

In [27]:
prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)
input_ids = assistant_tokenizer(prompt, return_tensors="pt").input_ids
gen_tokens = assistant_model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = assistant_tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

The discovery was made by scientists from the Universidad Nacional Mayor de San Miguel, in Colombia, led by Dr. Martin Parra from the Universidad de los Andes. The scientists found that the unicorns had a common ancestor and were descended from another


In [ ]:
first_output_from_test = "This rare valley, which is called the Uyuni City, is located some 250 miles north of the city of Cuzco, Peru. The city of Uyuni stands on the slopes of the mountain with a population of about 1,000 people"
second_output_from_test = "I am a lover of all things unicorn, and in my personal experience, unicorns are truly beautiful animals. \n In my personal experiences, they are just as beautiful as other animals. \n I am here to learn more about all of these animals, but we all need to talk"
third_output_from_test = "The study was published today in the journal PLOS ONE. \n The scientists discovered that the herd of unicorns had lived in the remote valley for hundreds of years. After studying three of the largest of the herd that could be spotted by the researchers, it was"
fourth_output_from_test = "The discovery was made by scientists from the Universidad Nacional Mayor de San Miguel, in Colombia, led by Dr. Martin Parra from the Universidad de los Andes. The scientists found that the unicorns had a common ancestor and were descended from another"
# Looks like it reliably produces different outputs. Where is randomness produced?

In [30]:
assistant_tokenizer.batch_decode(gen_tokens)

['In a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.\n\nThe discovery was made by scientists from the Universidad Nacional Mayor de San Miguel, in Colombia, led by Dr. Martin Parra from the Universidad de los Andes. The scientists found that the unicorns had a common ancestor and were descended from another']

## Load datasets

In [11]:
# Load webGPT comparisons dataset
dataset_webGPT = load_dataset("openai/summarize_from_feedback", "comparisons")
# dataset_harmless = load_dataset("Anthropic/hh-rlhf", data_dir="harmless-base")
# dataset = load_dataset("Anthropic/hh-rlhf", data_dir="red-team-attempts")

Found cached dataset summarize_from_feedback (/Users/peterchatain/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0)


  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
# Print the first 3 samples from the webGPT dataset
dataset_webGPT["train"][10]

{'info': {'id': 't3_ehyqt',
  'post': "A few months ago, I asked my RA if she wanted to participate in a fundraiser where we raced floaty toys down a river.  I asked her if she wanted to buy one because the winner won 50 dollars.  She signed up one for her and her sister.  It was her idea to buy one for her sister.  I paid for it because she didn't have cash.  Now fast forward to 2 months later, the second time I remind her that she still owes me money, and I have her rubber ducks.  I held onto the ducks until I got my money.\n\nShe just verbally assaulted me.  She denies ever wanting to participate in the toy race and says I pushed it on her, so I shouldn't be paid.  She was quite rude and disrespectful.  I'm calling bullshit because I would never sign someone up if they hadn't fully agreed.  It was her idea to include her sister!  \n\nNow she owes me 6 dollars and I'm angry.  I would have let it go despite the rudeness and disrespect.  I am now going to do everything in my power to e

In [24]:
dataset_webGPT["train"][10]['info']['post']

"A few months ago, I asked my RA if she wanted to participate in a fundraiser where we raced floaty toys down a river.  I asked her if she wanted to buy one because the winner won 50 dollars.  She signed up one for her and her sister.  It was her idea to buy one for her sister.  I paid for it because she didn't have cash.  Now fast forward to 2 months later, the second time I remind her that she still owes me money, and I have her rubber ducks.  I held onto the ducks until I got my money.\n\nShe just verbally assaulted me.  She denies ever wanting to participate in the toy race and says I pushed it on her, so I shouldn't be paid.  She was quite rude and disrespectful.  I'm calling bullshit because I would never sign someone up if they hadn't fully agreed.  It was her idea to include her sister!  \n\nNow she owes me 6 dollars and I'm angry.  I would have let it go despite the rudeness and disrespect.  I am now going to do everything in my power to ensure she pays me even if I have to bu

## Generate n (4) completions

In [ ]:
gpt_4_completions = {"train": {}}
for i in range(10):
  sample_post = dataset_webGPT["train"][i]['info']['post']
  gpt_4_completions["train"][i] = {"prompt": sample_post}
  gpt_4_completions["train"][i]["completions"] = []
  input_ids = assistant_tokenizer(sample_post, return_tensors="pt").input_ids
  for j in range(4):
    gen_tokens = assistant_model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=100,
    )
    completion = assistant_tokenizer.batch_decode(gen_tokens)[0]
    gpt_4_completions["train"][i]["completions"].append(completion)